In [41]:

import os
import numpy as np 
import pandas as pd 
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split

In [42]:
train_df = pd.read_csv("../input/train.csv")
train_df, val_df = train_test_split(train_df, test_size=0.1)

In [50]:
emb_index = {}
emb_file = open('../input/embeddings/glove.840B.300d/glove.840B.300d.txt')
for line in tqdm(emb_file):
    emb_values = line.split(" ")
    word = emb_values[0]
    vecs = np.asarray(emb_values[1:], dtype='float32')
    emb_index[word] = vecs
emb_file.close()

print('Found %s word vectors.' % len(emb_index))

2196017it [02:00, 18154.62it/s]

Found 2196016 word vectors.


In [79]:
TIME_STEP = 30
EMB_VEC_SIZE = 300
BATCH_SIZE = 128
def text_to_vec(text):
    init_emb = np.zeros(EMB_VEC_SIZE)
    text = text[:-1].split()[:TIME_STEP]
    embeds = [emb_index.get(x, init_emb) for x in text]
    embeds+= [init_emb] * (TIME_STEP - len(embeds))
    return np.array(embeds)


In [65]:

def train_gen(train_df, shuffle=False):
    n_batches = math.ceil(len(train_df) / BATCH_SIZE)
    while True: 
        if(shuffle==True):
            train_df = train_df.sample(frac=1.)  # Shuffle the data.
        for i in range(n_batches):
            texts = train_df.iloc[i*BATCH_SIZE:(i+1)*BATCH_SIZE, 1]
            text_arr = np.array([text_to_vec(text) for text in texts])
            yield text_arr, np.array(train_df["target"][i*BATCH_SIZE:(i+1)*BATCH_SIZE])

In [70]:
from keras.models import  Model
from keras.layers import LSTM, Dense, Bidirectional, Input, CuDNNLSTM
from tensorflow.python.client import device_lib

In [71]:
def get_available_gpus():
        local_device_protos = device_lib.list_local_devices()
        return [x.name for x in local_device_protos if x.device_type == 'GPU']
    
def get_base_model(input):

  num_gpus = get_available_gpus()
  #print(len(num_gpus))
  #Build LSTM network

  if(len(num_gpus)>0):
    first_lstm = Bidirectional(CuDNNLSTM(64, return_sequences=True, name='lstm1'))(input)
    second_lstm = Bidirectional(CuDNNLSTM(64, name='lstm2'))(first_lstm)
  else:  
    first_lstm = Bidirectional(LSTM(64, return_sequences=True, name='lstm1'))(input)
    second_lstm = Bidirectional(LSTM(64, name='lstm2'))(first_lstm)
  outputs = Dense(1, activation="sigmoid", name='last')(second_lstm)
  
  #create model LSTM+dense
  l_model = Model(input, outputs) 
  
  return(l_model)

In [72]:
features = Input(shape=(TIME_STEP, EMB_VEC_SIZE))
model = get_base_model(features)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

0


In [66]:

train_generator = train_gen(train_df, True)
validation_generator = train_gen(val_df, False)
model.fit_generator(train_generator,
                    steps_per_epoch=int(len(train_df) / BATCH_SIZE)
                    validation_data=validation_generator,
                    validation_steps = int(len(val_df) / BATCH_SIZE)
                    epochs=20,
                    verbose=True)

Epoch 1/1
500/500 [==============================] - 167s 333ms/step - loss: 0.1098 - acc: 0.9568 - val_loss: 0.1090 - val_acc: 0.9570
